In [ ]:
#Notebook for the paper titled "Shallow Planning with Partial Observability"

# Install required packages
!pip install numpy matplotlib pymdptoolbox joblib

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import mdptoolbox
import time
from joblib import Parallel, delayed

# Setting construction and partial observability structural parameters results

In [ ]:
class FixedMdp:
    """
    A class to represent a fixed Markov Decision Process (MDP).

    Attributes:
    -----------
    num_states : int
        Number of states in the MDP.
    num_actions : int
        Number of actions in the MDP.
    d : int
        Number of possible next states for each state-action pair.
    transitions : np.ndarray
        Transition probabilities for the MDP.
    rewards : np.ndarray
        Rewards for the MDP.

    Methods:
    --------
    generate_mdp():
        Generates the transition probabilities and rewards for an mdp of the type fixed. See the paper for details.
    """
    def __init__(self, num_states=10, d=3):
        self.num_states = num_states
        self.num_actions = 2
        self.d = d
        self.transitions = np.zeros((self.num_actions, num_states, num_states))
        self.rewards = np.zeros((num_states, self.num_actions))
        self.generate_mdp()

    def generate_mdp(self):
        for state in range(self.num_states):
            for action in range(self.num_actions):
                #Each state can be connected to d other states. We just fill non empty entries with transitions and rewards.
                probabilities = np.random.uniform(0, 1, self.d)
                probabilities /= probabilities.sum()
                next_states = np.random.choice(self.num_states, self.d, replace=False)
                self.rewards[state, action] = np.random.uniform(0, 1)
                self.transitions[action, state, next_states] = probabilities

def generate_mdps(num_mdps=1000, num_states=10, d=3):
    return [FixedMdp(num_states=num_states, d=d) for _ in range(num_mdps)]

In [ ]:
class RingMdp:
    """
    A class to represent a ring-shaped Markov Decision Process (MDP).

    Attributes:
    -----------
    num_states : int
        Number of states in the MDP.
    p : float
        Probability of additional random connections.
    num_actions : int
        Number of actions in the MDP.
    transitions : np.ndarray
        Transition probabilities for the MDP.
    rewards : np.ndarray
        Rewards for the MDP.

    Methods:
    --------
    generate_mdp():
        Generates the transition probabilities and rewards for an mdp of type ring. See the paper for details.
    """
    def __init__(self, num_states=10, p=0):
        self.num_states = num_states
        self.p = p
        self.num_actions = 2
        self.transitions = np.zeros((2, num_states, num_states))
        self.rewards = np.zeros((num_states, 2))
        self.generate_mdp()

    def generate_mdp(self):
        for state in range(self.num_states):
            next_state_clockwise = (state + 1) % self.num_states
            prev_state_counterclockwise = (state - 1) % self.num_states
            
            # Action 1: Move clockwise or stay
            self.transitions[0, state, [next_state_clockwise, state]] = np.random.uniform(0, 1, 2)

            # Action 2: Move counterclockwise or stay
            self.transitions[1, state, [prev_state_counterclockwise, state]] = np.random.uniform(0, 1, 2)
            
            # Additional random connections based on probability p
            for other_state in range(self.num_states):
                if other_state != state and other_state != next_state_clockwise and np.random.rand() < self.p:
                    self.transitions[0, state, other_state] = np.random.uniform(0, 1)
                if other_state != state and other_state != prev_state_counterclockwise and np.random.rand() < self.p:
                    self.transitions[1, state, other_state] = np.random.uniform(0, 1)
            
            # Normalize again after adding random connections
            self.transitions[0, state, :] /= self.transitions[0, state, :].sum()
            self.transitions[1, state, :] /= self.transitions[1, state, :].sum()
            
            # Generate rewards
            self.rewards[state, 0] = np.random.uniform(0, 1)
            self.rewards[state, 1] = np.random.uniform(0, 1)

def generate_ring_mdps(num_mdps=1000, num_states=10, p=0):
    return [RingMdp(num_states, p) for _ in range(num_mdps)]

In [ ]:
class POMDP:
    """
    Partially Observable Markov Decision Process (POMDP) class.

    Attributes
    ----------
    mdp : RingMdp
        The underlying Markov Decision Process (MDP).
    num_observations : int
        Number of possible observations.
    num_actions : int
        Number of possible actions.
    p_o_s : np.ndarray
        Observation probabilities given states.
    p_s_o : np.ndarray
        State probabilities given observations.
    transitions : np.ndarray
        Transition probabilities between observations.
    rewards : np.ndarray
        Rewards for each observation-action pair.

    Methods
    -------
    define_observation_prob():
        Defines the observation probabilities given states.
    define_p_s_o():
        Defines the state probabilities given observations.
    define_transitions():
        Defines the transition probabilities between observations.
    define_rewards():
        Defines the rewards for each observation-action pair.
    transition_function(current_state, action):
        Simulates a transition based on the MDP's transition probabilities.
    observation_function(next_state, action):
        Generates an observation based on the state's observation probability distribution.
    """
    def __init__(self, mdp, num_observations):
        """
        Initializes the POMDP with the given MDP and number of observations.

        Parameters
        ----------
        mdp : RingMdp
            The underlying Markov Decision Process (MDP).
        num_observations : int
            Number of possible observations.
        """
        self.mdp = mdp
        self.num_observations = num_observations
        self.num_actions = mdp.num_actions
        self.define_p_o_s()
        self.define_p_s_o()
        self.define_transitions()
        self.define_rewards()

    def define_p_o_s(self):
        """
        Defines the observation probabilities given states.
        """
        # Initialize P(O|S) for each state as a matrix of zeros
        self.p_o_s = np.zeros((self.mdp.num_states, self.num_observations))
        
        # Assign each state to an observation ensuring every observation has at least one state
        for state in range(self.mdp.num_states):
            self.p_o_s[state, state % self.num_observations] = 1

    def define_p_s_o(self):
        """
        Defines the state probabilities given observations.
        """
        # Initialize P(S|O) for each observation as a matrix of zeros
        self.p_s_o = np.zeros((self.num_observations, self.mdp.num_states))
        # Here for each observation we just define a uniform distribution on the corresponding ground states
        for obs in range(self.num_observations):
            ground_states = np.where(self.p_o_s[:, obs] == 1)[0]
            num_ground_states = len(ground_states)
            self.p_s_o[obs, :] = 0  # Set all probabilities to zero
            self.p_s_o[obs, ground_states] = 1 / num_ground_states  # Set uniform probability for relevant ground states
        
    def define_transitions(self):
        """
        Defines the transition probabilities between observations.
        """
        self.transitions = np.zeros((self.num_actions, self.num_observations, self.num_observations))

        for action in range(self.num_actions):
            for observation in range(self.num_observations):
                for next_observation in range(self.num_observations):
                    self.transitions[action, observation, next_observation] = np.sum([np.dot(self.p_s_o[observation, :], self.mdp.transitions[action, :, next_state]) for next_state in range(self.mdp.num_states) if self.p_s_o[next_observation, next_state] > 0])

    def define_rewards(self):
        """
        Defines the rewards for each observation-action pair.
        """
        self.rewards = np.einsum('os,sa->oa', self.p_s_o, self.mdp.rewards)

    def transition_function(self, current_state, action):
        """
        Simulates a transition based on the MDP's transition probabilities.

        Parameters
        ----------
        current_state : int
            The current state in the MDP.
        action : int
            The action taken.

        Returns
        -------
        int
            The next state in the MDP.
        """
        return np.random.choice(self.mdp.num_states, p=self.mdp.transitions[action, current_state])

In [ ]:
# This cell sets up all the POMDPs (Partially Observable Markov Decision Processes) for the simulations.
# It first generates a list of MDPs (Markov Decision Processes) using the generate_mdps function (use fixed or ring).
mdps = generate_mdps(num_mdps=10000, num_states=10, d=3)

# Define a list of omega values which will be used to create POMDPs with varying cardinality of the observation space (hence the name Omega).
omega_values = [10, 8, 6, 4, 2, 1]

# Initialize a 2D list to store all the POMDPs. The outer list corresponds to each MDP,
# and the inner list corresponds to each omega value for that MDP.
all_pomdps = [[None for _ in omega_values] for _ in mdps]

# Iterate over each MDP and each omega value to create the corresponding POMDP.
for mdp_idx, mdp in enumerate(mdps):
    for p_idx, p in enumerate(omega_values):
        pomdp = POMDP(mdp, p)
        all_pomdps[mdp_idx][p_idx] = pomdp

In [ ]:
#This cell calculates the action variation for each POMDPs and each underlying mdps.

delta_M_o = np.zeros((len(all_pomdps), len(omega_values)))
for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
    for p_idx, p in enumerate(omega_values):
        for o in range(pomdp_instance[p_idx].num_observations):
            if np.sum(np.abs(pomdp_instance[p_idx].transitions[0,o,:] - pomdp_instance[p_idx].transitions[1,o,:])) > delta_M_o[pomdp_idx, p_idx]:
                delta_M_o[pomdp_idx, p_idx] = np.sum(np.abs(pomdp_instance[p_idx].transitions[0,o,:] - pomdp_instance[p_idx].transitions[1,o,:]))

delta_M_s = np.zeros((len(all_pomdps)))
for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
    for s in range(pomdp_instance[0].mdp.num_states):
        if np.sum(np.abs(pomdp_instance[0].mdp.transitions[0,s,:] - pomdp_instance[0].mdp.transitions[1,s,:])) > delta_M_s[pomdp_idx]:
            delta_M_s[pomdp_idx] = np.sum(np.abs(pomdp_instance[0].mdp.transitions[0,s,:] - pomdp_instance[0].mdp.transitions[1,s,:]))

In [ ]:
#normalize by dividing the parameter on each POMDP by the one on the state space.
normalized_delta_M_o = np.zeros((len(all_pomdps), len(omega_values)))
for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
    for p_idx, p in enumerate(omega_values):
        normalized_delta_M_o[pomdp_idx, p_idx] = delta_M_o[pomdp_idx, p_idx] / delta_M_s[pomdp_idx]

In [ ]:
#This cell calculates the value-function variation for each POMDPs and their underlying MDPs.
gammas = np.linspace(0.0001,0.99,3)

state_values = np.zeros((len(gammas),len(all_pomdps), all_pomdps[0][0].mdp.num_states))
#kappa_s
kappa_gamma_s = np.zeros((len(gammas), len(all_pomdps)))
for gamma_idx, gamma in enumerate(gammas):
    for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
        pi = mdptoolbox.mdp.PolicyIteration(pomdp_instance[0].mdp.transitions, pomdp_instance[0].mdp.rewards,discount=gamma)
        pi.run()
        kappa_gamma_s[gamma_idx, pomdp_idx] = max(pi.V) - min(pi.V)
        state_values[gamma_idx, pomdp_idx, :] = pi.V


kappa_gamma_phi = np.zeros((len(gammas), len(all_pomdps), len(omega_values)))
#for each pomdps find the optimal values for each p
for gamma_idx, gamma in enumerate(gammas):
    for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
        for p_idx, p in enumerate(omega_values):
            pi = mdptoolbox.mdp.PolicyIteration(pomdp_instance[p_idx].transitions, pomdp_instance[p_idx].rewards,discount=gamma)
            pi.run()
            kappa_gamma_phi[gamma_idx, pomdp_idx, p_idx] = max(pi.V) - min(pi.V)

In [ ]:
#This cell calculates the bias for every POMDPs for varying values of gamma.

#You can set up the precision by changing the number of points in the linspace as long as the discount factor is between ]0,1[.
gammas_biases = np.linspace(0.0001,0.99,100)

bias = np.zeros((len(gammas_biases), len(all_pomdps), len(omega_values)))

# Now calculate the relative losses
for p_idx, p in enumerate(omega_values):
    for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
        for gamma_idx, gamma in enumerate(gammas_biases):
            if gamma_idx == 0:
                pi = mdptoolbox.mdp.PolicyIteration(pomdp_instance[p_idx].transitions, pomdp_instance[p_idx].rewards, discount=gammas_biases[-1])
                pi.run()
                Max_values = pi.V
            pi = mdptoolbox.mdp.PolicyIteration(pomdp_instance[p_idx].transitions, pomdp_instance[p_idx].rewards, discount=gamma)
            pi.run()
            policy_array = np.array(pi.policy)
            R_pi = pomdp_instance[p_idx].rewards[np.arange(pomdp_instance[p_idx].num_observations), policy_array]
            P_pi = pomdp_instance[p_idx].transitions[policy_array, np.arange(pomdp_instance[p_idx].num_observations), :]
            V_pi_lowgamm = np.linalg.inv(np.eye(pomdp_instance[p_idx].num_observations) - gammas_biases[-1] * P_pi) @ R_pi
            bias[gamma_idx, pomdp_idx, p_idx] = np.max((Max_values - V_pi_lowgamm) / Max_values)



In [ ]:
#Normalize the value-function variation for each POMDPs by the value of the parameter on the underlying MDP.
normalized_kappa_gamma = np.zeros((len(gammas), len(all_pomdps), len(omega_values)))
for gamma_idx, gamma in enumerate(gammas):
    for pomdp_idx, pomdp_instance in enumerate(all_pomdps):
        for p_idx, p in enumerate(omega_values):
            normalized_kappa_gamma[gamma_idx, pomdp_idx, p_idx] = kappa_gamma_phi[gamma_idx, pomdp_idx, p_idx] / kappa_gamma_s[gamma_idx, pomdp_idx]


In [ ]:
# This cell calculates the Blackwell optimal discount factor gamma for each POMDP and omega value.
# See the paper for a detailed explanation of the algorithm.
gammas_for_bw = np.linspace(0.99, 0.0001, 100)  

blackwell_gammas = np.zeros((len(all_pomdps), len(omega_values)))

def process_pomdp(pomdp_instance, p_idx, p, gammas):
    last_policy = None
    for gamma_idx, gamma in enumerate(gammas_for_bw):
        pi = mdptoolbox.mdp.PolicyIteration(pomdp_instance[p_idx].transitions, pomdp_instance[p_idx].rewards, discount=gamma)
        pi.run()
        if last_policy is not None and not np.array_equal(pi.policy, last_policy):
            break
        last_policy = pi.policy
        gamma_value = gamma
    return gamma_value

#We use Joblib to paralellize this process since it can be very slow and every POMDPs are independant.
results = Parallel(n_jobs=-1)(delayed(process_pomdp)(pomdp_instance, p_idx, p, gammas) for pomdp_idx, pomdp_instance in enumerate(all_pomdps) for p_idx, p in enumerate(omega_values))

#Bringing the results in the right data structure.
for idx, result in enumerate(results):
    pomdp_idx = idx // len(omega_values)
    p_idx = idx % len(omega_values)
    blackwell_gammas[pomdp_idx, p_idx] = result

In [ ]:
#This cell plots the data for the blackwell discount factor and the bias which varies as a function of Omega
fig, axs = plt.subplots(1, 2, figsize=(11, 4), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.25})  # Create a figure with 1 row and 2 columns, increased figure width and added space between plots

# Common settings for all plots
plot_settings = {
    'showmeans': True,
    'showextrema': True,
    'showmedians': True
}

#Plotting the blackwell discount factor
axs[0].violinplot(blackwell_gammas, positions=range(len(omega_values)), showmeans=True, showextrema=True, showmedians=True)
axs[0].set_xticks(range(len(omega_values)))
axs[0].set_xticklabels(omega_values)
axs[0].set_ylabel('$\gamma_{Bw}$', fontsize=11)
axs[0].set_xlabel('Number of abstracted states $|\Omega|$', fontsize=11)


#Plotting the bias
for p_idx, p in enumerate(omega_values):
    axs[1].plot(gammas_biases, bias[:, :, p_idx].mean(axis=1), label=p)
axs[1].set_ylabel('Average normalized bias', fontsize=11)
axs[1].legend(title="$|\Omega|$", fontsize='small')  
axs[1].set_xlabel('$\gamma$', fontsize=11)


# Save the figure as a PDF file
plt.savefig("Combined_Plots_bias.pdf", format='pdf')
plt.show()
plt.close()


In [ ]:
# This cell generates the plots for the normalized value-function variation and the normalized action variation for different values of
# Omega.

fig, axs = plt.subplots(1, 2, figsize=(11, 4))  # Create a figure with 1 row and 2 columns

#We start by plotting the normalized value-function variation for different values of Omega.
violin_parts = []
offset_increment = 0.2

for i, gamma in enumerate(gammas):
    positions = [p + i * offset_increment for p in range(len(omega_values))]  
    vp = axs[0].violinplot(normalized_kappa_gamma[i], positions=positions, **plot_settings, widths=0.3)  
    violin_parts.append(vp)

axs[0].set_xticks([p + (len(gammas) - 1) * 0.2 / 2 for p in range(len(omega_values))])  
axs[0].set_xticklabels(omega_values)
axs[0].set_ylabel(r'Normalized value-function variation $\frac{\kappa_{M,\gamma}^\phi}{\kappa_{M,\gamma}^S}$', fontsize=11)
axs[0].set_xlabel('Number of abstracted states $|\Omega|$', fontsize=11)
labels = [f'$\gamma$ = {gamma:.2f}' for gamma in gammas]
axs[0].legend([vp["bodies"][0] for vp in violin_parts], labels, loc='upper right', fontsize=10)


#Next we plot the normalized action variation for different values of Omega.
axs[1].violinplot(normalized_delta_M_o, positions=range(len(omega_values)), **plot_settings)
axs[1].set_xticks(range(len(omega_values)))
axs[1].set_xticklabels(omega_values)
axs[1].set_ylabel(r'Normalized action variation $\frac{\delta_M^\phi}{\delta_M^S}$', fontsize=11)
axs[1].set_yticks(np.arange(0, 1.1, 0.5))
axs[1].set_xlabel('Number of abstracted states $|\Omega|$', fontsize=11)

plt.savefig("Combined_Plots_params.pdf", format='pdf')
plt.show()
plt.close()
